In [52]:
import numpy as np
from numpy import linalg as LA
from scipy.io import loadmat
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import cm
import matplotlib as mpl
import cv2
import computer_vision as cv
from icecream import ic
from tqdm import trange
import time
from get_dataset_info import *

# %load_ext snakeviz
# %matplotlib inline
%matplotlib qt
%config InlineBackend.figure_format = 'retina'
from matplotlib import rc
rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)

In [53]:
def estimate_E_robust_old(K, x1_norm, x2_norm, n_its, n_samples, err_threshold_px, alpha):
    
    err_threshold = err_threshold_px / K[0,0]
    best_inliers = None
    best_E = None
    max_inliers = 0
    epsilon = 0
    T = n_its
    n_points = x1_norm.shape[1]

    for t in trange(n_its):

        rand_mask = np.random.choice(np.size(x1_norm,1), n_samples, replace=False)
        E = cv.estimate_E_DLT(x1_norm[:,rand_mask], x2_norm[:,rand_mask], enforce=True, verbose=False)

        D1, D2 = cv.compute_epipolar_errors(E, x1_norm, x2_norm)
        inliers = ((D1**2 + D2**2) / 2) < err_threshold**2

        n_inliers = np.sum(inliers)

        if n_inliers > max_inliers:
            best_inliers = np.copy(inliers)
            best_E = np.copy(E)
            max_inliers = n_inliers
            print('No. inliers:', np.sum(inliers), end='\r')

            # Extract R and T

            # epsilon = max_inliers / n_points
            # T = cv.compute_ransac_iterations(alpha, epsilon, n_samples)
            # print('New T:', T, 'New epsilon:', epsilon)
            # if t >= 4*T-1:
            #     print('Bailout at iteration:', t, T)
            #     break
        
    return best_E, best_inliers

In [54]:
def compute_valid_inliers(P1, P2, X, inliers):

    x1_norm_valid = P1 @ X
    x2_norm_valid = P2 @ X
    valid_coords_P1 = x1_norm_valid[-1,:] > 0
    valid_coords_P2 = x2_norm_valid[-1,:] > 0
    valid_coords = valid_coords_P1 * valid_coords_P2
    valid_inliers = inliers * valid_coords

    return valid_inliers

In [55]:
def estimate_H_DLT(img1_pts, img2_pts, verbose=False):

    n = np.size(img1_pts,1)
    M = []

    for i in range(n):

        x = img1_pts[0,i]
        y = img1_pts[1,i]

        u = img2_pts[0,i]
        v = img2_pts[1,i]

        m = np.array([[x, y, 1, 0, 0, 0, -u*x, -u*y, -u],
                      [0, 0, 0, x, y, 1, -v*x, -v*y, -v]])

        M.append(m)

    M = np.concatenate(M, 0)
    U, S, VT = LA.svd(M, full_matrices=False)
    H = np.stack([VT[-1, i:i+3] for i in range(0, 9, 3)], 0)

    if verbose:
        M_approx = U @ np.diag(S) @ VT
        v = VT[-1,:] # last row of VT because optimal v should be last column of V
        Mv = M @ v
        print('\n||Mv||:', (Mv @ Mv)**0.5)
        print('||v||^2:', v @ v)
        print('max{||M - M_approx||}:', np.max(np.abs(M - M_approx)))
        print('S:', S)

    return H

In [56]:
def homography_to_RT(H, x1, x2):
    def unitize(a, b):
        denom = 1.0 / np.sqrt(a**2 + b**2)
        ra = a * denom
        rb = b * denom
        return ra, rb

    # Check the right sign for H
    if LA.det(H) < 0:
        H *= -1 
        
    N = x1.shape[1]
    if x1.shape[0] != 3:
        x1 = np.vstack([x1, np.ones((1, N))])
    if x2.shape[0] != 3:
        x2 = np.vstack([x2, np.ones((1, N))])

    positives = np.sum(np.sum(x2 * (H @ x1), axis=0) > 0)
    if positives < (N / 2):
        H *= -1

    U, S, VT = np.linalg.svd(H, full_matrices=False)
    V = VT.T
    s1 = S[0] / S[1]
    s3 = S[2] / S[1]
    zeta = s1 - s3
    a1 = np.sqrt(1 - s3**2)
    b1 = np.sqrt(s1**2 - 1)
    a, b = unitize(a1, b1)
    c, d = unitize(1+s1*s3, a1*b1)
    e, f = unitize(-b/s1, -a/s3)
    v1, v3 = V[:, 0], V[:, 2]
    n1 = b * v1 - a * v3
    n2 = b * v1 + a * v3
    R1 = U @ np.array([[c, 0, d], [0, 1, 0], [-d, 0, c]]) @ VT
    R2 = U @ np.array([[c, 0, -d], [0, 1, 0], [d, 0, c]]) @ VT
    t1 = e * v1 + f * v3
    t2 = e * v1 - f * v3
    if n1[2] < 0:
        t1 = -t1
        n1 = -n1
    if n2[2] < 0:
        t2 = -t2
        n2 = -n2

    # Move from Triggs' convention H = R*(I - t*n') to H&Z notation H = R - t*n'
    t1 = R1 @ t1
    t2 = R2 @ t2

    # Verify that we obtain the initial homography back
    # H /= S[1]
    # print(np.linalg.norm(R1 - zeta * np.outer(t1, n1) - H), np.linalg.norm(R2 - zeta * np.outer(t2, n2) - H))

    return R1, t1, R2, t2

# Example usage:
# H is the homography matrix, x1 and x2 are the corresponding 2D points
# R1, t1, R2, t2 = homography_to_RT(H, x1, x2)

In [57]:
def compute_point_point_distance(x_proj, x_img):
    D = LA.norm(x_proj - x_img, axis=0)
    return D

In [58]:
def compute_E_validity(E):
    rank = LA.matrix_rank(E)
    valid = True if rank == 2 else False
    return valid

In [59]:
def compute_E_inliers(E, E_valid, x1_norm, x2_norm, err_threshold, epsilon_E):

    best_E = None
    best_inliers = None
    best_epsilon = epsilon_E
     
    if E_valid:
        D1, D2 = cv.compute_epipolar_errors(E, x1_norm, x2_norm)
        inliers = ((D1**2 + D2**2) / 2) < err_threshold**2
        n_inliers = np.sum(inliers)
        epsilon = n_inliers / x1_norm.shape[1]

        if epsilon > best_epsilon:
            best_E = np.copy(E)
            best_inliers = np.copy(inliers)
            best_epsilon = epsilon
    
    return best_E, best_inliers, best_epsilon

In [60]:
# P1 = cv.get_canonical_camera()
# best_P2 = None
# best_X = None
# max_inliers = 0
# if epsilon > epsilon_E:

#     # P2_arr = cv.extract_P_from_E(E)
#     # X_arr = cv.compute_triangulated_X_from_extracted_P2_solutions(P1, P2_arr, x1_norm, x2_norm)
#     # P2_valid, X_valid = cv.extract_valid_camera_and_points(P1, P2_arr, X_arr)
#     # valid_inliers = compute_valid_inliers(P1, P2_valid, X_valid, inliers)
#     # n_valid_inliers = np.sum(valid_inliers)
#     # print(n_valid_inliers, n_inliers)

#     # best_P2 = np.copy(P2_valid)
#     # best_X = np.copy(X_valid)
#     best_E = np.copy(E)
#     best_inliers = np.copy(inliers)
#     epsilon_E = epsilon
#     print('No. inliers:', n_inliers, end='\r')

#     # if n_valid_inliers > max_inliers:
#     #     best_inliers = np.copy(valid_inliers)
#     #     best_P2 = np.copy(P2_valid)
#     #     max_inliers = n_valid_inliers
#     #     print('No. valid inliers:', n_valid_inliers, end='\r')

#         # epsilon = max_inliers / n_points
#         # T = cv.compute_ransac_iterations(alpha, epsilon, n_samples)
#         # print('New T:', T, 'New epsilon:', epsilon)
#         # if t >= 4*T-1:
#         #     print('Bailout at iteration:', t, T)
#         #     break

# valid_inliers = compute_valid_inliers(P1, best_P2, best_X, best_inliers)
# n_valid_inliers = np.sum(valid_inliers)
# print('No. valid inliers:', n_valid_inliers, 'No. inliers:', np.sum(best_inliers))


In [61]:
def verbose_E_robust(t, T_E, T_H, epsilon_E, epsilon_H, inliers, method):
    print('Iteration:', t, 'T_E:', T_E, 'T_H:', T_H, 'epsilon_E:', np.round(epsilon_E, 2), 'epsilon_H:', np.round(epsilon_H, 2), 'No. inliers:', np.sum(inliers), 'From:', method)

In [72]:
def estimate_E_robust(K, x1_norm, x2_norm, min_its, max_its, scale_its, alpha, err_threshold_px):
    
    err_threshold = err_threshold_px / K[0,0]
    best_E = None
    best_inliers = None
    n_points = x1_norm.shape[1]
    n_E_samples = 8
    n_H_samples = 4
    epsilon_E = 0
    epsilon_H = 0
    T_E = max_its
    T_H = max_its

    t = 0
    while t < T_E and t < T_H:
        t += 1

        rand_mask = np.random.choice(np.size(x1_norm,1), n_E_samples, replace=False)
        E = cv.estimate_E_DLT(x1_norm[:,rand_mask], x2_norm[:,rand_mask], enforce=True, verbose=False)
        E_valid = compute_E_validity(E)

        E_trial, inliers_trial, epsilon_E_trial = compute_E_inliers(E, E_valid, x1_norm, x2_norm, err_threshold, epsilon_E)
        if E_trial is not None:
            best_E = np.copy(E_trial)
            best_inliers = np.copy(inliers_trial)
            epsilon_E = epsilon_E_trial
            T_E = cv.compute_ransac_iterations(alpha, epsilon_E, n_E_samples, min_its, max_its, scale_its)
            verbose_E_robust(t, T_E, T_H, epsilon_E, epsilon_H, best_inliers, method='E 8-point alg.')
        
        rand_mask = np.random.choice(np.size(x1_norm,1), n_H_samples, replace=False)
        H = estimate_H_DLT(x1_norm[:,rand_mask], x2_norm[:,rand_mask], verbose=False)
        x2_norm_proj = cv.dehomogenize(H @ x1_norm)
        D = compute_point_point_distance(x2_norm_proj, x2_norm)
        inliers = D**2 < err_threshold**2
        n_inliers = np.sum(inliers)
        epsilon_H_trial = n_inliers / n_points

        if epsilon_H_trial > epsilon_H:
            
            # num, Rs, Ts, Ns = cv2.decomposeHomographyMat(H, np.eye(3))
            R1, T1, R2, T2 = homography_to_RT(H, x1_norm, x2_norm)
            E1 = cv.compute_E_from_R_and_T(R1, T1)
            E2 = cv.compute_E_from_R_and_T(R2, T2)

            E1_valid = compute_E_validity(E1)
            E2_valid = compute_E_validity(E2)

            E_trial, inliers_trial, epsilon_E_trial = compute_E_inliers(E1, E1_valid, x1_norm, x2_norm, err_threshold, epsilon_E)
            if E_trial is not None:
                best_E = np.copy(E_trial)
                best_inliers = np.copy(inliers_trial)
                epsilon_E = epsilon_E_trial
                epsilon_H = epsilon_H_trial
                T_E = cv.compute_ransac_iterations(alpha, epsilon_E, n_H_samples, min_its, max_its, scale_its)
                T_H = cv.compute_ransac_iterations(alpha, epsilon_H, n_H_samples, min_its, max_its, scale_its)
                verbose_E_robust(t, T_E, T_H, epsilon_E, epsilon_H, best_inliers, method='H 4-point alg.')

            E_trial, inliers_trial, epsilon_E_trial = compute_E_inliers(E2, E2_valid, x1_norm, x2_norm, err_threshold, epsilon_E)
            if E_trial is not None:
                best_E = np.copy(E_trial)
                best_inliers = np.copy(inliers_trial)
                epsilon_E = epsilon_E_trial
                epsilon_H = epsilon_H_trial
                T_E = cv.compute_ransac_iterations(alpha, epsilon_E, n_H_samples, min_its, max_its, scale_its)
                T_H = cv.compute_ransac_iterations(alpha, epsilon_H, n_H_samples, min_its, max_its, scale_its)
                verbose_E_robust(t, T_E, T_H, epsilon_E, epsilon_H, best_inliers, method='H 4-point alg.')

    print('Bailout at iteration:', t)
    return best_E, best_inliers

In [63]:
data_set = 0
K, img_names, init_pair, pixel_threshold = get_dataset_info(data_set)
K_inv = LA.inv(K)
imgs = cv.load_image(img_names, multi=True)
n_imgs = imgs.shape[0]
n_camera_pairs = n_imgs - 1
img1_init = imgs[init_pair[0]]
img2_init = imgs[init_pair[1]]
sift = False

In [64]:
# Compute and save SIFT points
marg = 0.7

if sift:
    
    # SIFT points for relative orientation
    for i in range(n_camera_pairs):
        print("\nCamera pair:", i+1, "/", n_camera_pairs)
        img1 = imgs[i]
        img2 = imgs[i+1]
        x1, x2, _, _, _, _, _ = cv.compute_sift_points(img1, img2, marg)
        np.save('data/dataset_{}_RO_x1_{}.npy'.format(data_set, i), x1)
        np.save('data/dataset_{}_RO_x2_{}.npy'.format(data_set, i), x2)
        

    # SIFT points for camera resection
    x1, x2, kp1, kp2, des1, des2, _ = cv.compute_sift_points(img1_init, img2_init, marg)
    np.save('data/dataset_{}_CR_x1_{}.npy'.format(data_set, init_pair[1]), x1)
    np.save('data/dataset_{}_CR_x2_{}.npy'.format(data_set, init_pair[1]), x2)

    for i in range(n_imgs):

        if i != init_pair[0] and i != init_pair[1]:
            
            print("\nImage:", i+1, "/", n_imgs)
            img2 = imgs[i]
            x1, x2 = cv.compute_sift_points_sequential(kp1, des1, img2, marg)
            np.save('data/dataset_{}_CR_x1_{}.npy'.format(data_set, i), x1)
            np.save('data/dataset_{}_CR_x2_{}.npy'.format(data_set, i), x2)

In [73]:
# Load SIFT points

# SIFT points for relative orientation
x1s_norm_RO = []
x2s_norm_RO = []

for i in range(n_camera_pairs):

    x1 = np.load('data/dataset_{}_RO_x1_{}.npy'.format(data_set, i))
    x2 = np.load('data/dataset_{}_RO_x2_{}.npy'.format(data_set, i))
    x1_norm = cv.dehomogenize(K_inv @ x1)
    x2_norm = cv.dehomogenize(K_inv @ x2)
    x1s_norm_RO.append(x1_norm)
    x2s_norm_RO.append(x2_norm)

x1s_norm_RO = np.array(x1s_norm_RO)
x2s_norm_RO = np.array(x2s_norm_RO)


# SIFT points for camera resectioning
x1s_norm_CR = []
x2s_norm_CR = []

for i in range(n_imgs):

    if i != init_pair[0]:

        x1 = np.load('data/dataset_{}_CR_x1_{}.npy'.format(data_set, i))
        x2 = np.load('data/dataset_{}_CR_x2_{}.npy'.format(data_set, i))
        x1_norm = cv.dehomogenize(K_inv @ x1)
        x2_norm = cv.dehomogenize(K_inv @ x2)
        x1s_norm_CR.append(x1_norm)
        x2s_norm_CR.append(x2_norm)

x1s_norm_CR = np.array(x1s_norm_CR)
x2s_norm_CR = np.array(x2s_norm_CR)

In [74]:
# Compute absolute rotations

min_its = 0
max_its = 5000
scale_its = 1
alpha = 0.95
P1 = cv.get_canonical_camera()
abs_rots = [P1[:,:-1]]
abs_trans = [P1[:,-1]]

for i in range(n_camera_pairs):    
    
    x1_norm = x1s_norm_RO[i]
    x2_norm = x2s_norm_RO[i]
    E, inliers = estimate_E_robust(K, x1_norm, x2_norm, min_its, max_its, scale_its, alpha, pixel_threshold)
    x1_norm_inliers = x1_norm[:,inliers]
    x2_norm_inliers = x2_norm[:,inliers]

    P2_arr = cv.extract_P_from_E(E)
    X_arr = cv.compute_triangulated_X_from_extracted_P2_solutions(P1, P2_arr, x1_norm_inliers, x2_norm_inliers)
    P2, _ = cv.extract_valid_camera_and_points(P1, P2_arr, X_arr)

    R1 = abs_rots[i]
    T1 = abs_trans[i]
    R2 = P2[:,:-1] @ R1
    T2 = P2[:,-1] + (R2 @ R1.T @ T1)

    abs_rots.append(R2)
    abs_trans.append(T2)


c:\Users\erikn\skola\EEN020-Computer-Vision\project\computer_vision.py:347: RuntimeWarning: divide by zero encountered in scalar divide
  T = scale * np.ceil(np.log(1-alpha) / np.log(1-epsilon**s))


Iteration: 1 T_E: 5000 T_H: 5000 epsilon_E: 0.0 epsilon_H: 0 No. inliers: 8 From: E 8-point alg.
Iteration: 1 T_E: 5000 T_H: 5000 epsilon_E: 0.02 epsilon_H: 0.0 No. inliers: 409 From: H 4-point alg.
Iteration: 2 T_E: 5000 T_H: 5000 epsilon_E: 0.14 epsilon_H: 0.0 No. inliers: 2384 From: E 8-point alg.
Iteration: 283 T_E: 5000 T_H: 5000 epsilon_E: 0.24 epsilon_H: 0.0 No. inliers: 3996 From: E 8-point alg.
Iteration: 983 T_E: 5000 T_H: 5000 epsilon_E: 0.27 epsilon_H: 0.0 No. inliers: 4443 From: E 8-point alg.
Iteration: 1638 T_E: 157.0 T_H: 5000 epsilon_E: 0.61 epsilon_H: 0.0 No. inliers: 10053 From: E 8-point alg.
Bailout at iteration: 1638
No. valid coords for each camera pair: [    0 20106 10053 10053]
Argmax(P2_arr): 1


In [68]:
# Reconstruct initial 3D points

x1_init_norm = x1s_norm_CR[init_pair[1]-1]
x2_init_norm = x2s_norm_CR[init_pair[1]-1]
E, inliers = estimate_E_robust(K, x1_init_norm, x2_init_norm, min_its, max_its, scale_its, alpha, pixel_threshold)
np.save('data/dataset_{}_CR_inliers_{}.npy'.format(data_set, init_pair[1]), inliers)

Iteration: 1 T_E: 2000 T_H: 2000 epsilon_E: 0.01 epsilon_H: 0 No. inliers: 128 From: E 8-point alg.
Iteration: 2 T_E: 2000 T_H: 2000 epsilon_E: 0.04 epsilon_H: 0 No. inliers: 592 From: E 8-point alg.
Iteration: 11 T_E: 2000 T_H: 2000 epsilon_E: 0.04 epsilon_H: 0 No. inliers: 656 From: E 8-point alg.
Iteration: 80 T_E: 2000 T_H: 2000 epsilon_E: 0.04 epsilon_H: 0 No. inliers: 672 From: E 8-point alg.
Iteration: 86 T_E: 2000 T_H: 2000 epsilon_E: 0.05 epsilon_H: 0 No. inliers: 872 From: E 8-point alg.
Iteration: 129 T_E: 2000 T_H: 2000 epsilon_E: 0.12 epsilon_H: 0 No. inliers: 1909 From: E 8-point alg.
Iteration: 148 T_E: 2000 T_H: 2000 epsilon_E: 0.4 epsilon_H: 0 No. inliers: 6574 From: E 8-point alg.
Iteration: 1556 T_E: 1648.0 T_H: 2000 epsilon_E: 0.45 epsilon_H: 0 No. inliers: 7516 From: E 8-point alg.
Bailout at iteration: 1648


In [69]:
inliers = np.load('data/dataset_{}_CR_inliers_{}.npy'.format(data_set, init_pair[1]))
x1_init_norm_inliers = x1_init_norm[:,inliers]
x2_init_norm_inliers = x2_init_norm[:,inliers]

P2_arr = cv.extract_P_from_E(E)
X_arr = cv.compute_triangulated_X_from_extracted_P2_solutions(P1, P2_arr, x1_init_norm_inliers, x2_init_norm_inliers)
P2, X = cv.extract_valid_camera_and_points(P1, P2_arr, X_arr)

R1_init = abs_rots[init_pair[0]]
T1_init = abs_trans[init_pair[0]]
X = R1_init.T @ X[:-1,:] # - T1_init[:,np.newaxis] # Which way of rotating is correct? What about translation?

No. valid coords for each camera pair: [    0 15032  7516  7516]
Argmax(P2_arr): 1


In [ ]:
# Find camera matrices



In [70]:
def plot_cameras_and_3D_points(X, C_arr, axis_arr, s, path, save=False):
    
    fig = plt.figure()
    ax = plt.axes(projection='3d')

    ax.plot(X[0], X[1], X[2], '.', ms=1, color='magenta', label='Est. X')
    cv.plot_cameras_and_axes(ax, C_arr, axis_arr, s)

    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    ax.set_zlabel('$z$')
    ax.set_aspect('equal')
    ax.view_init(elev=-50, azim=-104, roll=20)
    ax.legend(loc="lower right")
    fig.tight_layout()
    if save:
        fig.savefig(path, dpi=300)
    plt.show()

In [71]:
P1 = np.column_stack((abs_rots[init_pair[0]], abs_trans[init_pair[0]]))
P2 = np.column_stack((abs_rots[init_pair[1]], abs_trans[init_pair[1]]))
P_arr = np.array([P1, P2])
C_arr, axis_arr = cv.compute_camera_center_and_normalized_principal_axis(P_arr, multi=True)
s = 2
plot_cameras_and_3D_points(X, C_arr, axis_arr, s, None, save=False)

In [16]:
alpha = 0.5
epsilon = 0.2
s = 8
np.ceil(np.log(1-alpha) / np.log(1-epsilon**s))

270761.0

In [12]:
plt_3D = True
save = False

x = np.linspace(0,10,11)
y = np.random.rand(11)
fig = plt.figure()
plt.plot(x,y)
fig.savefig('report-images/test.png', bbox_inches='tight')
img = cv.load_image('report-images/test.png')
plt.imshow(img)